In [1]:
import hash_blocking_spark as hash
import ngram_blocking_spark as ngram
import matchers_spark as match
import similarity_spark as sim
import blocking_structured_and_sorted_spark as ss
import cluster_spark as cluster

In [2]:
from pyspark.sql import SparkSession
import pandas as pd

dblp_csv = '../CSV-files/dblp_stem.csv'
dblp = pd.read_csv(dblp_csv)

acm_csv = '../CSV-files/acm_stem.csv'
acm = pd.read_csv(acm_csv)

selected_columns = ['paper_title']
spark = SparkSession.builder.appName("sparkk").getOrCreate()
acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)


# this was in our experiment in two the best match
a = hash.initial_hash_parallel_df(spark,acm_df, selected_columns)
b = hash.initial_hash_parallel_df(spark, dblp_df, selected_columns)
c = match.apply_similarity_blocks_spark(a,b, 0.7, sim.jaccard_similarity)

Processing time: 32.16172432899475 seconds. Number of index combinations: 1052


In [3]:
import csv

def write_index_pairs_to_csv(data, filename):
    header = ['dblp_index', 'acm_index']
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header
        for row in data:
            cleaned_row = [row[0].strip('[]'), row[1].strip('[]')]  # Remove brackets
            writer.writerow(cleaned_row)

write_index_pairs_to_csv(c, '../Matched/Matched_spark.csv')
 

In [4]:
# get matched entites as a list back for spark and without
import pandas as pd
def reconstructed_pairs(path):
    df_pairs = pd.read_csv(path)
    return list(zip(df_pairs['dblp_index'], df_pairs['acm_index']))

pairs = reconstructed_pairs('../Matched/Matched Entities.csv')
pairs_spark = reconstructed_pairs('../Matched/Matched_spark.csv')

In [5]:
# check if they are equal
def pairs_correspond(pair_list1, pair_list2):
    sorted_pair_list1 = [tuple(sorted(pair)) for pair in pair_list1]
    sorted_pair_list2 = [tuple(sorted(pair)) for pair in pair_list2]

    for pair1 in sorted_pair_list1:
        if pair1 not in sorted_pair_list2:
            return False
    return True

pairs_correspond(pairs_spark, pairs)

True

In [6]:
import csv
# cluster the spark pairs which are equals
def cluster_to_csv(cluster_data, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for idx, item in enumerate(cluster_data, start=1):
            writer.writerow([f'Cluster {idx}: {item}'])

cluster_pairs = cluster.build_clusters_parallel(pairs_spark)
cluster_to_csv(cluster_pairs, '/Users/aliaslan/Documents/GitHub/DIA-ER/Matched/Spark_Cluster.csv')
print(len(cluster_pairs))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/aliaslan/Documents/GitHub/DIA-ER/Matched/Spark_Cluster.csv'

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, lower, reverse, col
from pyspark.sql.types import StringType
from itertools import combinations

dblp_csv = '../CSV-files/dblp_stem.csv'
dblp = pd.read_csv(dblp_csv)

acm_csv = '../CSV-files/acm_stem.csv'
acm = pd.read_csv(acm_csv)
selected_columns = ['paper_title']

@udf(StringType())
def alphanumerical(entity: str) -> str:
    return ''.join([char for char in entity if char.isalnum()])
@udf(StringType())
def title(entity: str) -> str:
    return entity.title()
@udf(StringType())
def identity(entity: str) -> str:
    return entity

augementations = [identity, lower, title, alphanumerical, reverse]
for i in range(1, len(augementations)):
    print(f"Blocking paper_title with {i} augmentations.\n")
    for comb in enumerate(combinations(augementations, i)):
        spark = SparkSession.builder.appName("sparkk").getOrCreate()
        acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
        dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)
        dblp_result_df = dblp_df.withColumn("paper_title", comb[1](comb[0](col("paper_title"))))
        acm_result_df = acm_df.withColumn("paper_title", comb[1](comb[0](col("paper_title"))))
        c = hash.initial_hash_parallel_df(spark,acm_df, selected_columns)
        d = hash.initial_hash_parallel_df(spark, dblp_df, selected_columns)
        e = match.apply_similarity_blocks_spark(c,d, 0.7, sim.jaccard_similarity)
        print(f"Blocking paper_title with {comb[0].__name__} and {comb[1].__name__} resulted in {len(e)} matches.")
        spark.stop

Processing time: 33.05843901634216 seconds. Number of index combinations: 1052
Blocking paper_title with identity and lower resulted in 1052 matches.
Processing time: 34.25920796394348 seconds. Number of index combinations: 1052
Blocking paper_title with identity and title resulted in 1052 matches.
Processing time: 37.75804615020752 seconds. Number of index combinations: 1052
Blocking paper_title with identity and alphanumerical resulted in 1052 matches.
Processing time: 35.127676010131836 seconds. Number of index combinations: 1052
Blocking paper_title with identity and reverse resulted in 1052 matches.
Processing time: 34.83160614967346 seconds. Number of index combinations: 1052
Blocking paper_title with lower and title resulted in 1052 matches.


In [ ]:
# Example that other are functioning

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkk").getOrCreate()

acm_df = spark.read.csv(acm_csv, header=True, inferSchema=True)
dblp_df = spark.read.csv(dblp_csv, header=True, inferSchema=True)

selected_columns = ['author_names', 'paper_title']
c = ngram.initial_ngram_parallel_df(spark,acm_df, selected_columns,2)
d = ngram.initial_ngram_parallel_df(spark, dblp_df, selected_columns,2)
e = match.apply_similarity_blocks_spark(c,d, 0.7, sim.jaccard_similarity)



ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

Processing time: 1.8916327953338623 seconds. Number of index combinations: 782


<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x11b3e8310>>